# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import json

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [2]:
url = 'https://scanner.tradingview.com/america/scan'
payload = {"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"exchange","operation":"in_range","right":["AMEX","NASDAQ","NYSE"]},{"left":"is_primary","operation":"equal","right":True},{"left":"active_symbol","operation":"equal","right":True}],"options":{"lang":"en"},"markets":["america"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","change|1","change|5","change|15","change|60","change|240","change","change|1W","change|1M","Perf.3M","Volatility.W","average_volume_90d_calc","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","currency","fundamental_currency_code"],"sort":{"sortBy":"name","sortOrder":"asc"},"range":[0,8033]}
data = requests.post(url, json = payload)

In [3]:
stocks_data = data.json()['data']
len(stocks_data)

8033

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
stocks_data[0]

{'s': 'NYSE:A',
 'd': ['agilent-technologies',
  'A',
  140.9,
  -0.02128565,
  0.05787181,
  0.09945301,
  0.29181495,
  0.62125107,
  0.40618542,
  -2.10519002,
  -0.75368036,
  -8.75534257,
  2.36723958,
  1311818.36666667,
  'Agilent Technologies, Inc.',
  'stock',
  'common',
  'delayed_streaming_900',
  100,
  1,
  'false',
  0,
  'USD',
  'USD']}

In [5]:
stats_from_tradingview = []

for data in stocks_data:
    stock = {}
    stock['Symbol'] = data['d'][1]
    stock['Name'] = data['d'][14]
    stock['Price'] = data['d'][2]
    stock['1min CHG'] = data['d'][3]
    stock['5min CHG'] = data['d'][4]
    stock['15min CHG'] = data['d'][5]
    stock['1Hr CHG'] = data['d'][6]
    stock['4Hr CHG'] = data['d'][7]
    stock['1Day CHG'] = data['d'][8]
    stock['1Week CHG'] = data['d'][9]
    stock['1Month CHG'] = data['d'][10]
    stock['3Month CHG'] = data['d'][11]
    stock['Volatility'] = data['d'][12]
    stock['AvgVolume'] = data['d'][13]
    
    stats_from_tradingview.append(stock)

In [6]:
stats_from_tradingview[0]['5min CHG']

0.05787181

In [7]:
# Serializing json
json_object = json.dumps(stats_from_tradingview)
 
# Writing to sample.json
with open("stocks_all_performance_short_term.json", "w") as outfile:
    outfile.write(json_object)

In [8]:
hqm_dataframe = pd.read_json('stocks_all_performance_short_term.json')
available_stocks = pd.read_json('curated_stocks_all.json')
available_stocks_list = available_stocks['symbol'].tolist()

In [9]:
hqm_dataframe = hqm_dataframe[hqm_dataframe['Symbol'].isin(available_stocks_list)]

In [10]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume
0,A,"Agilent Technologies, Inc.",140.90,-0.021286,0.057872,0.099453,0.291815,0.621251,0.406185,-2.105190,-0.753680,-8.755343,2.367240,1.311818e+06
1,AA,Alcoa Corporation,51.02,-0.048976,-0.254154,0.098097,0.810117,0.810117,1.694240,-8.105187,4.250102,5.435007,4.640770,5.140235e+06
9,AAMC,Altisource Asset Management Corp Com,58.00,2.745793,2.745793,2.745793,2.745793,2.745793,1.736537,9.022351,11.517016,150.000000,5.309126,9.140722e+03
10,AAME,Atlantic American Corporation,2.37,-0.420168,-3.658537,-3.658537,-3.658537,-0.836820,-0.836820,-0.832671,-5.200000,-9.541985,4.830502,4.066700e+03
11,AAN,"Aarons Holdings Company, Inc.",12.16,0.164609,0.453983,0.828500,1.416667,1.163757,-0.815661,-4.851330,-15.261324,0.996678,6.955606,3.335110e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021,ZS,"Zscaler, Inc.",114.95,-0.173686,-0.138997,-0.381316,0.753791,-0.303556,-2.485579,-3.597786,-12.352268,0.559881,4.618574,2.683018e+06
8026,ZTS,Zoetis Inc.,168.64,0.020757,-0.077023,-0.145061,0.225828,0.202008,0.963899,-1.125704,0.982036,12.030824,1.757424,2.337952e+06
8027,ZUMZ,Zumiez Inc.,23.28,-0.257180,-0.021482,0.042992,0.605275,1.173913,2.781457,-1.146497,0.085985,1.129453,3.575307,3.606098e+05
8028,ZUO,"Zuora, Inc.",9.68,-0.308960,-0.360268,-0.051626,0.207039,0.000000,-0.308960,-3.393214,14.285714,68.935428,6.447703,1.164694e+06


In [11]:
mean_val = hqm_dataframe['AvgVolume'].mean()
hqm_dataframe['AvgVolume'].fillna(value=mean_val, inplace=True)
mean_val = hqm_dataframe['1Week CHG'].mean()
hqm_dataframe['1Week CHG'].fillna(value=mean_val, inplace=True)
hqm_dataframe.isna().any()

Symbol        False
Name          False
Price         False
1min CHG      False
5min CHG      False
15min CHG     False
1Hr CHG       False
4Hr CHG       False
1Day CHG      False
1Week CHG     False
1Month CHG    False
3Month CHG    False
Volatility    False
AvgVolume     False
dtype: bool

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Cleaning Data
Clean all rows that have no historical Price Return, as in Nonetype

In [12]:
# for row in hqm_dataframe.index:
#     if(hqm_dataframe.loc[row, '1yearPerformace'] == None):
#         hqm_dataframe.drop(row, inplace=True)

hqm_dataframe.reset_index(inplace=True, drop=True)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume
0,A,"Agilent Technologies, Inc.",140.90,-0.021286,0.057872,0.099453,0.291815,0.621251,0.406185,-2.105190,-0.753680,-8.755343,2.367240,1.311818e+06
1,AA,Alcoa Corporation,51.02,-0.048976,-0.254154,0.098097,0.810117,0.810117,1.694240,-8.105187,4.250102,5.435007,4.640770,5.140235e+06
2,AAMC,Altisource Asset Management Corp Com,58.00,2.745793,2.745793,2.745793,2.745793,2.745793,1.736537,9.022351,11.517016,150.000000,5.309126,9.140722e+03
3,AAME,Atlantic American Corporation,2.37,-0.420168,-3.658537,-3.658537,-3.658537,-0.836820,-0.836820,-0.832671,-5.200000,-9.541985,4.830502,4.066700e+03
4,AAN,"Aarons Holdings Company, Inc.",12.16,0.164609,0.453983,0.828500,1.416667,1.163757,-0.815661,-4.851330,-15.261324,0.996678,6.955606,3.335110e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,ZS,"Zscaler, Inc.",114.95,-0.173686,-0.138997,-0.381316,0.753791,-0.303556,-2.485579,-3.597786,-12.352268,0.559881,4.618574,2.683018e+06
3155,ZTS,Zoetis Inc.,168.64,0.020757,-0.077023,-0.145061,0.225828,0.202008,0.963899,-1.125704,0.982036,12.030824,1.757424,2.337952e+06
3156,ZUMZ,Zumiez Inc.,23.28,-0.257180,-0.021482,0.042992,0.605275,1.173913,2.781457,-1.146497,0.085985,1.129453,3.575307,3.606098e+05
3157,ZUO,"Zuora, Inc.",9.68,-0.308960,-0.360268,-0.051626,0.207039,0.000000,-0.308960,-3.393214,14.285714,68.935428,6.447703,1.164694e+06


In [14]:
list(hqm_dataframe.columns.values)

['Symbol',
 'Name',
 'Price',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
 '1Month CHG',
 '3Month CHG',
 'Volatility',
 'AvgVolume']

In [15]:
time_periods = [
    '1min CHG',
    '5min CHG',
    '15min CHG',
    '1Hr CHG',
    '4Hr CHG',
    '1Day CHG',
    '1Week CHG',
#     '1Month CHG',
#     '3Month CHG',
    'Volatility'
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period}'
        percentile_col = f'{time_period} Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,...,Volatility,AvgVolume,1min CHG Percentile,5min CHG Percentile,15min CHG Percentile,1Hr CHG Percentile,4Hr CHG Percentile,1Day CHG Percentile,1Week CHG Percentile,Volatility Percentile
0,A,"Agilent Technologies, Inc.",140.90,-0.021286,0.057872,0.099453,0.291815,0.621251,0.406185,-2.105190,...,2.367240,1.311818e+06,34.251345,60.240582,52.231719,39.664451,56.473568,62.266540,52.801519,28.110161
1,AA,Alcoa Corporation,51.02,-0.048976,-0.254154,0.098097,0.810117,0.810117,1.694240,-8.105187,...,4.640770,5.140235e+06,28.711618,16.904084,52.041785,76.194998,66.191833,84.013928,9.718265,65.590377
2,AAMC,Altisource Asset Management Corp Com,58.00,2.745793,2.745793,2.745793,2.745793,2.745793,1.736537,9.022351,...,5.309126,9.140722e+03,99.018677,98.733777,98.258943,97.657487,95.853118,84.393796,97.657487,72.111428
3,AAME,Atlantic American Corporation,2.37,-0.420168,-3.658537,-3.658537,-3.658537,-0.836820,-0.836820,-0.832671,...,4.830502,4.066700e+03,8.198797,0.379867,0.601456,0.823045,12.440646,30.864198,69.452358,67.679645
4,AAN,"Aarons Holdings Company, Inc.",12.16,0.164609,0.453983,0.828500,1.416667,1.163757,-0.815661,-4.851330,...,6.955606,3.335110e+05,77.397911,88.414055,90.946502,91.199747,78.600823,31.085787,22.981956,83.792339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,ZS,"Zscaler, Inc.",114.95,-0.173686,-0.138997,-0.381316,0.753791,-0.303556,-2.485579,-3.597786,...,4.618574,2.683018e+06,15.321304,25.387781,13.738525,73.599240,19.119975,13.295347,33.713200,65.305476
3155,ZTS,Zoetis Inc.,168.64,0.020757,-0.077023,-0.145061,0.225828,0.202008,0.963899,-1.125704,...,1.757424,2.337952e+06,52.358341,32.225388,24.343147,35.106046,35.612536,73.820829,65.938588,9.718265
3156,ZUMZ,Zumiez Inc.,23.28,-0.257180,-0.021482,0.042992,0.605275,1.173913,2.781457,-1.146497,...,3.575307,3.606098e+05,11.839190,40.930674,45.742324,64.925609,79.012346,91.896170,65.685343,52.928142
3157,ZUO,"Zuora, Inc.",9.68,-0.308960,-0.360268,-0.051626,0.207039,0.000000,-0.308960,-3.393214,...,6.447703,1.164694e+06,10.541311,12.693890,31.560620,33.618234,27.793606,42.196898,35.992403,80.848370


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [16]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)   

In [17]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,...,AvgVolume,1min CHG Percentile,5min CHG Percentile,15min CHG Percentile,1Hr CHG Percentile,4Hr CHG Percentile,1Day CHG Percentile,1Week CHG Percentile,Volatility Percentile,HQM Score
0,A,"Agilent Technologies, Inc.",140.90,-0.021286,0.057872,0.099453,0.291815,0.621251,0.406185,-2.105190,...,1.311818e+06,34.251345,60.240582,52.231719,39.664451,56.473568,62.266540,52.801519,28.110161,48.254986
1,AA,Alcoa Corporation,51.02,-0.048976,-0.254154,0.098097,0.810117,0.810117,1.694240,-8.105187,...,5.140235e+06,28.711618,16.904084,52.041785,76.194998,66.191833,84.013928,9.718265,65.590377,49.920861
2,AAMC,Altisource Asset Management Corp Com,58.00,2.745793,2.745793,2.745793,2.745793,2.745793,1.736537,9.022351,...,9.140722e+03,99.018677,98.733777,98.258943,97.657487,95.853118,84.393796,97.657487,72.111428,92.960589
3,AAME,Atlantic American Corporation,2.37,-0.420168,-3.658537,-3.658537,-3.658537,-0.836820,-0.836820,-0.832671,...,4.066700e+03,8.198797,0.379867,0.601456,0.823045,12.440646,30.864198,69.452358,67.679645,23.805002
4,AAN,"Aarons Holdings Company, Inc.",12.16,0.164609,0.453983,0.828500,1.416667,1.163757,-0.815661,-4.851330,...,3.335110e+05,77.397911,88.414055,90.946502,91.199747,78.600823,31.085787,22.981956,83.792339,70.552390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,ZS,"Zscaler, Inc.",114.95,-0.173686,-0.138997,-0.381316,0.753791,-0.303556,-2.485579,-3.597786,...,2.683018e+06,15.321304,25.387781,13.738525,73.599240,19.119975,13.295347,33.713200,65.305476,32.435106
3155,ZTS,Zoetis Inc.,168.64,0.020757,-0.077023,-0.145061,0.225828,0.202008,0.963899,-1.125704,...,2.337952e+06,52.358341,32.225388,24.343147,35.106046,35.612536,73.820829,65.938588,9.718265,41.140393
3156,ZUMZ,Zumiez Inc.,23.28,-0.257180,-0.021482,0.042992,0.605275,1.173913,2.781457,-1.146497,...,3.606098e+05,11.839190,40.930674,45.742324,64.925609,79.012346,91.896170,65.685343,52.928142,56.619975
3157,ZUO,"Zuora, Inc.",9.68,-0.308960,-0.360268,-0.051626,0.207039,0.000000,-0.308960,-3.393214,...,1.164694e+06,10.541311,12.693890,31.560620,33.618234,27.793606,42.196898,35.992403,80.848370,34.405666


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [18]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe.reset_index(inplace=True, drop=True)
# hqm_dataframe[hqm_dataframe['Price'] < 10][:50]

In [19]:
cols = list(hqm_dataframe.columns.values)
cols

['Symbol',
 'Name',
 'Price',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
 '1Month CHG',
 '3Month CHG',
 'Volatility',
 'AvgVolume',
 '1min CHG Percentile',
 '5min CHG Percentile',
 '15min CHG Percentile',
 '1Hr CHG Percentile',
 '4Hr CHG Percentile',
 '1Day CHG Percentile',
 '1Week CHG Percentile',
 'Volatility Percentile',
 'HQM Score']

In [20]:
# rearrange columns
cols = list(hqm_dataframe.columns.values)
hqm_dataframe = hqm_dataframe[['Symbol',
 'Name',
 'Price',
 'HQM Score',
 'AvgVolume',
 'Volatility',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
#  '1Month CHG',
#  '3Month CHG',
 '1min CHG Percentile',
 '5min CHG Percentile',
 '15min CHG Percentile',
 '1Hr CHG Percentile',
 '4Hr CHG Percentile',
 '1Day CHG Percentile',
 '1Week CHG Percentile',
#  '1Month CHG Percentile',
#  '3Month CHG Percentile',
 'Volatility Percentile'
]]

In [21]:
hqm_dataframe[:50]

,Symbol,Name,Price,HQM Score,AvgVolume,Volatility,1min CHG,5min CHG,15min CHG,1Hr CHG,...,1Day CHG,1Week CHG,1min CHG Percentile,5min CHG Percentile,15min CHG Percentile,1Hr CHG Percentile,4Hr CHG Percentile,1Day CHG Percentile,1Week CHG Percentile,Volatility Percentile
0,LABP,"Landos Biopharma, Inc.",0.3800,98.330168,5.917798e+05,20.410645,4.614953,19.442665,18.871533,17.043315,...,22.501612,1.387407,99.746755,100.000000,99.968344,99.968344,99.968344,99.873378,87.685977,99.430199
1,NXPL,NextPlat Corp,1.7500,97.317189,2.524454e+04,9.714606,0.574713,1.156069,2.941176,4.166667,...,13.636364,15.894040,92.276037,96.169674,98.654638,99.208610,99.905033,99.635961,99.113644,93.573916
2,HOUR,"Hour Loop, Inc.",2.3900,97.202437,1.942876e+04,8.570467,6.696429,6.696429,6.696429,6.701192,...,3.017241,5.286344,99.936689,99.873378,99.778411,99.778411,99.651789,93.130738,95.251662,90.218424
3,ACGN,"Aceragen, Inc.",4.6300,97.194524,3.159504e+04,8.388517,3.003337,3.003337,3.003337,3.003337,...,5.949657,9.715640,99.271921,99.081988,98.765432,98.100665,96.549541,98.100665,97.942387,89.743590
4,CDXC,ChromaDex Corporation,1.8300,97.071858,2.278406e+05,6.387296,3.314917,5.056180,6.851037,6.857143,...,12.962963,9.580838,99.398544,99.746755,99.841722,99.810066,99.810066,99.588477,97.847420,80.531814
5,XELA,"Exela Technologies, Inc.",0.0689,97.044160,1.083423e+08,23.554562,0.747384,11.958273,23.217550,25.046382,...,12.032520,0.437318,94.143716,99.968344,100.000000,100.000000,100.000000,99.525166,83.127572,99.588477
6,TAOP,Taoping Inc.,0.7699,95.777936,4.807918e+04,9.316199,1.838624,1.838624,1.838624,1.838624,...,5.465753,10.001586,98.163976,97.784109,96.676163,94.713517,90.440013,97.720798,98.005698,92.719215
7,DYAI,"Dyadic International, Inc.",1.7400,95.623615,4.295081e+04,10.873308,1.162791,1.162791,1.363160,1.363160,...,2.958580,17.567568,96.644508,96.201330,95.346629,90.345046,98.702121,92.972460,99.335233,95.441595
8,ENVB,"Enveric Biosciences, Inc.",1.9800,95.522713,3.936142e+04,7.298890,3.125000,3.125000,3.125000,3.125000,...,4.210526,2.061856,99.335233,99.176955,98.923710,98.290598,96.866097,96.296296,90.075973,85.216841
9,DXR,Daxor Corporation,9.2900,95.322887,6.823000e+03,5.447341,2.993348,2.993348,2.993348,6.171429,...,6.171429,5.209513,99.240266,99.050332,98.733777,99.715100,99.525166,98.163976,95.156695,72.997784
